In [1]:
# import sys
# !{sys.executable} -m pip install numpy
# !{sys.executable} -m pip install tensorflow
# !{sys.executable} -m pip install keras
# import time

## Libraries

In [2]:
import csv
import string
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from numpy import array
from keras.layers import Dropout, Dense, Embedding, LSTM
from keras import Input
from keras.layers.merge import add
from keras.models import Model
import os, numpy as np

Using TensorFlow backend.


In [3]:
datasetFolder = "../FoodIgredients/"
datasetName = "ingredients v1.csv"
datasetPath = datasetFolder + datasetName

## Read Data

In [4]:


dataset = []
with open(datasetPath, mode="r", encoding="utf-8-sig") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    keys = []
    for row in csv_reader:
        if line_count == 0:
            keys = row
            line_count += 1
        else:
            record = dict(zip(keys, row))
            dataset+= [record]
    print(f'Processed {len(dataset)} lines.')

print(dataset[0])

Processed 10000 lines.
{'id': 'AVphBRHOilAPnD_x0OrE', 'asins': 'B00HXST15C', 'brand': 'Simon Fischer', 'categories': 'Grocery & Gourmet Food,Food,Grocery', 'dateAdded': '2017-01-07T20:13:17Z', 'dateUpdated': '2017-06-30T16:48:02Z', 'ean': '41642026706', 'features.key': 'Ingredients', 'features.value': 'Dried Prunes,Water,Corn Syrup,Sugar,Pectin.', 'manufacturer': 'Sokol And Company', 'manufacturerNumber': '33829', 'name': 'Simon Fischer Fruit Bttr Prune Lekvar', 'sizes': '', 'upc': '41642026706', 'weight': '10.6 pounds', '': ''}


## Filtering Food which doesn't has ingredients

In [5]:
dataset = [row for row in dataset 
           if row["features.key"].lower() == "ingredients"]
print(dataset[0])

{'id': 'AVphBRHOilAPnD_x0OrE', 'asins': 'B00HXST15C', 'brand': 'Simon Fischer', 'categories': 'Grocery & Gourmet Food,Food,Grocery', 'dateAdded': '2017-01-07T20:13:17Z', 'dateUpdated': '2017-06-30T16:48:02Z', 'ean': '41642026706', 'features.key': 'Ingredients', 'features.value': 'Dried Prunes,Water,Corn Syrup,Sugar,Pectin.', 'manufacturer': 'Sokol And Company', 'manufacturerNumber': '33829', 'name': 'Simon Fischer Fruit Bttr Prune Lekvar', 'sizes': '', 'upc': '41642026706', 'weight': '10.6 pounds', '': ''}


## get food label, ingredients tuple

In [6]:
finalDataset = [
    {"Name":record["name"], "Ingredients": record["features.value"].strip(".").split(",")}
    for record in dataset 
    if len(record["features.value"].strip(".").split(",")) < 50 and 
    2 < len(record["features.value"].strip(".").split(",")) and
    len(record["name"].split()) < 6]
print(finalDataset[0])

{'Name': 'Jolly Time Popcorn', 'Ingredients': ['Salt', ' Yellow 5 Lake', ' Tricalcium Phosphate And Artificial Butter Flavor']}


In [7]:
ingredients = [
    ['startseq'] + [ing.lower().strip() for ing in rec["Ingredients"]] + ['endseq']
    for rec in finalDataset]
food = [rec["Name"].lower().strip() for rec in finalDataset]
print(food[0])
print(ingredients[0])

jolly time popcorn
['startseq', 'salt', 'yellow 5 lake', 'tricalcium phosphate and artificial butter flavor', 'endseq']


## Cleaning Data

In [8]:

# prepare translation table for removing punctuation
table = str.maketrans('', '', string.punctuation)
for i in range(len(ingredients)):
    key = ingredients[i]
    for j in range(len(key)):
        desc = key[j]
        # tokenize
        desc = desc.split()
        # convert to lower case
        desc = [word.lower() for word in desc]
        # remove punctuation from each token
        desc = [w.translate(table) for w in desc]
        # remove hanging 's' and 'a'
        desc = [word for word in desc if len(word)>1]
        # remove tokens with numbers in them
        desc = [word for word in desc if word.isalpha()]
        # store as string
        key[j] = ' '.join(desc)
print(ingredients[0])

['startseq', 'salt', 'yellow lake', 'tricalcium phosphate and artificial butter flavor', 'endseq']


## Original & Distinct Vocabulary Space

In [9]:
allVocab = [i for ing in ingredients for i in ing]
vocabulary = set(allVocab)
print("All Vocabulary = %d" % len(allVocab))
print("Distinct Vocabulary = %d" % len(vocabulary))

All Vocabulary = 26893
Distinct Vocabulary = 5525


## Freq-based Filtering

In [10]:
# Create a list of all the training captions
all_train_ingreds = allVocab
# for key, val in train_descriptions.items():
#     for cap in val:
#         all_train_captions.append(cap)

# Consider only words which occur at least 10 times in the corpus
word_count_threshold = 10
word_counts = {}
nIngreds = 0
for ingred in all_train_ingreds:
    nIngreds += 1
#     for w in ingred.split(' '):
    w = ingred
    word_counts[w] = word_counts.get(w, 0) + 1

vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]

print(vocab[0])
print('preprocessed words %d ' % len(vocab))
# preprocessed words 1651

startseq
preprocessed words 346 


## Vocabulary Encoding

In [11]:
encVocab = {"<UNK>": 1, "<PAD>": 0}
for ingred in vocab: # list of list of ingredients
    for token in ingred.split():
        if token not in encVocab.keys():
            encVocab[token] = len(encVocab)
    if ingred not in encVocab.keys():
        encVocab["_".join(ingred.split())] = len(encVocab)
        
for f in food:
    if f not in encVocab.keys():
        encVocab["_".join(f.split())] = len(encVocab)
    for token in f.split():
        if token not in encVocab.keys():
            encVocab[token] = len(encVocab)
print("Number of unique tokens: {}".format(len(encVocab)))
vocab_size = len(encVocab)

Number of unique tokens: 4853


In [12]:
wordtoix = {}
ixtoword = {}
for word, ix in encVocab.items():
    wordtoix[word] = ix
    ixtoword[ix] = word


## Determine the maximum ingredients count per meal

In [13]:
allIngreds = ingredients

def maxCount(allIngreds):
    mx = max(len(ingreds) for ingreds in allIngreds)
    return mx

max_count = maxCount(allIngreds)
print("Maximum Ingredients count = %d" % max_count)

Maximum Ingredients count = 51


## Data Generator

In [14]:

# data generator, intended to be used in a call to model.fit_generator()
def data_generator(ingredients, foods, wordtoix, max_length, num_foods_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over foods
    while 1:
        for idx, ingreds in enumerate(ingredients):
            n+=1
            # retrieve the food feature
            food = foods[idx]
            # encode the sequence
            seq = [wordtoix[word] for word in ingreds if word in wordtoix]
            
            # split one sequence into multiple X, y pairs
            for i in range(1, len(seq)):
                # split into input and output pair
                in_seq, out_seq = seq[:i], seq[i]
                # pad input sequence
                in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                # encode output sequence
                out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                # store
                X1.append(food)
                X2.append(in_seq)
                y.append(out_seq)
            # yield the batch data
            if n==num_foods_per_batch:
                yield [[array(X1), array(X2)], array(y)]
                X1, X2, y = list(), list(), list()
                n=0

## Word Embeddings

In [15]:
%%time
# Load Glove vectors
glove_dir = '../dataset/glove'
embeddings_index = {} # empty dictionary
f = open(os.path.join(glove_dir, 'glove.6B.200d.txt'), encoding="utf-8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

CPU times: user 40.1 s, sys: 1.05 s, total: 41.2 s
Wall time: 42.4 s


In [16]:
embedding_dim = 200# Get 200-dim dense vector for each of the 10000 words in out vocabulary
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for ingred, i in wordtoix.items():
    #if i < max_words:
    # else np.zeros((200,))
    embedding_vector = np.sum([embeddings_index.get(word) for word in ingred.split("_") if embeddings_index.get(word) is not None], axis = 0)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector

# 12. Model Architecture

In [17]:
cleanIngredsList = [" ".join(["_".join(ingred.split()) for ingred in ingreds]) for ingreds in ingredients]

cleanFood = ["_".join(f.split()) for f in food]
food_vocab = list(set(cleanFood))
food_vocab_size = len(food_vocab)

print("Food Vocab size: %d" % food_vocab_size)
print(cleanFood[0])
print(cleanIngredsList[0])

Food Vocab size: 1702
jolly_time_popcorn
startseq salt yellow_lake tricalcium_phosphate_and_artificial_butter_flavor endseq


In [39]:


# image feature extractor model
inputs1 = Input(shape=(200,))
# fe0 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs1)
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)

# partial caption sequence model
inputs2 = Input(shape=(max_count,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)

# decoder (feed forward) model
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)

# merge the two input models
model = Model(inputs=[inputs1, inputs2], outputs=outputs)

In [40]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 51)           0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 51, 200)      970600      input_4[0][0]                    
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 200)          0           input_3[0][0]                    
____________________________________________________________________________________________

### set embedding layer to untrainable

In [41]:
model.layers[2]

In [42]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False

# model.layers[3].set_weights([embedding_matrix])
# model.layers[3].trainable = False

### compile the model using adam optimizer

In [43]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [44]:
epochs = 10
number_pics_per_bath = 3
# training_food_ingredients = dict(zip(cleanFood, cleanIngredsList))
steps = len(cleanIngredsList)//number_pics_per_bath

In [45]:
%time
foodEmbeddings = np.zeros((len(food), embedding_dim))

for i, f in enumerate(food):
    embedding_vector = np.sum([embeddings_index.get(word) for word in f.split("_") if embeddings_index.get(word) is not None], axis = 0)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        foodEmbeddings[i] = embedding_vector


CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 16.2 µs


In [46]:
# dont run
import random

validCount = int(len(cleanIngredsList) * 0.33)

validIndices = random.sample(range(len(cleanIngredsList)), validCount)
trainIndeces = [i for i in range(len(cleanIngredsList)) if i not in validIndices]

cleanIngredsList_valid = [cleanIngredsList[i] for i in validIndices]
cleanIngredsList_train = [cleanIngredsList[i] for i in trainIndeces]
foodEmbeddings_valid = [foodEmbeddings[i] for i in validIndices]
foodEmbeddings_train = [foodEmbeddings[i] for i in trainIndeces]

train_generator = data_generator(cleanIngredsList_train, foodEmbeddings_train, wordtoix, max_count, number_pics_per_bath)
valid_generator = data_generator(cleanIngredsList_valid, foodEmbeddings_valid, wordtoix, max_count, number_pics_per_bath)
# print(len(cleanIngredsList))
# print(int(len(cleanIngredsList) * 0.33))
# print(len(cleanIngredsList) - int(len(cleanIngredsList) * 0.33))

In [47]:
%%time
for i in range(epochs):
#     generator = data_generator(cleanIngredsList, foodEmbeddings, wordtoix, max_count, number_pics_per_bath)
    train_generator = data_generator(cleanIngredsList_train, foodEmbeddings_train, wordtoix, max_count, number_pics_per_bath)
    valid_generator = data_generator(cleanIngredsList_valid, foodEmbeddings_valid, wordtoix, max_count, number_pics_per_bath)
    model.fit_generator(generator=train_generator, validation_data=valid_generator, validation_steps=steps, epochs=1, steps_per_epoch=steps, verbose=1)
    model.save('./model_weights/model_' + str(i) + '.h5')
    

Epoch 1/1
584/584 [==============================] - 655s 1s/step - loss: 1.4422 - accuracy: 0.4808 - val_loss: 1.2317 - val_accuracy: 0.4568
Epoch 1/1
584/584 [==============================] - 650s 1s/step - loss: 1.2697 - accuracy: 0.4940 - val_loss: 1.1908 - val_accuracy: 0.4724
Epoch 1/1
584/584 [==============================] - 652s 1s/step - loss: 1.2547 - accuracy: 0.5014 - val_loss: 1.1874 - val_accuracy: 0.4958
Epoch 1/1
584/584 [==============================] - 646s 1s/step - loss: 1.2429 - accuracy: 0.5043 - val_loss: 1.1881 - val_accuracy: 0.5004
Epoch 1/1
584/584 [==============================] - 669s 1s/step - loss: 1.2339 - accuracy: 0.5072 - val_loss: 1.1823 - val_accuracy: 0.5032
Epoch 1/1
584/584 [==============================] - 727s 1s/step - loss: 1.2253 - accuracy: 0.5096 - val_loss: 1.1802 - val_accuracy: 0.5080
Epoch 1/1
584/584 [==============================] - 753s 1s/step - loss: 1.2145 - accuracy: 0.5144 - val_loss: 1.1751 - val_accuracy: 0.5083
Epoch 

In [ ]:
%%time
for i in range(epochs):
    train_generator = data_generator(cleanIngredsList_train, foodEmbeddings_train, wordtoix, max_count, number_pics_per_bath)
    valid_generator = data_generator(cleanIngredsList_valid, foodEmbeddings_valid, wordtoix, max_count, number_pics_per_bath)
    model.fit_generator(generator=train_generator, validation_data=valid_generator, validation_steps=steps, epochs=1, steps_per_epoch=steps, verbose=1)
    model.save('./model_weights/model_' + str(i) + '.h5')

Epoch 1/1
584/584 [==============================] - 647s 1s/step - loss: 1.1585 - accuracy: 0.5340 - val_loss: 1.1522 - val_accuracy: 0.5018
Epoch 1/1
584/584 [==============================] - 647s 1s/step - loss: 1.1379 - accuracy: 0.5401 - val_loss: 1.1670 - val_accuracy: 0.5014
Epoch 1/1
584/584 [==============================] - 677s 1s/step - loss: 1.1116 - accuracy: 0.5505 - val_loss: 1.1558 - val_accuracy: 0.4947
Epoch 1/1
108/584 [====>.........................] - ETA: 5:49 - loss: 1.0427 - accuracy: 0.5724

In [ ]:
model.optimizer.learning_rate = 0.0001

In [ ]:
# model.optimizer.lr = 0.0001
epochs = 10
number_pics_per_bath = 6
steps = len(cleanIngredsList)//number_pics_per_bath

In [ ]:
import keras
new_model = keras.models.load_model('./model_weights/model_9.h5')

for i in range(epochs):    
    train_generator = data_generator(cleanIngredsList_train, foodEmbeddings_train, wordtoix, max_count, number_pics_per_bath)
    valid_generator = data_generator(cleanIngredsList_valid, foodEmbeddings_valid, wordtoix, max_count, number_pics_per_bath)
    model.fit_generator(generator=train_generator, validation_data=valid_generator, validation_steps=steps, epochs=1, steps_per_epoch=steps, verbose=1)
    #model.save('./model_weights/model_' + str(i) + '.h5')

In [28]:
model.save_weights('./model_weights/model_30.h5')

# Prediction

Load Model

In [29]:
model.load_weights('./model_weights/model_30.h5')

Greedy Prediction Function

In [52]:
max_length = max_count
def greedySearch(food):
    in_text = 'startseq'
#     in_text = ''
    for i in range(max_length):
        sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
        sequence = pad_sequences([sequence], maxlen=max_length)
        print(food.shape)
        print(sequence.shape)
        yhat = model.predict([food,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
#   in_text in_text.strip()
        if word == 'endseq':
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

In [53]:
z= 15
key = z
foodEmbedding = foodEmbeddings[key]
print(foodEmbedding.shape)

print("Label: %s" % cleanFood[key])
print("Actual Ingredients: ", cleanIngredsList[key])
print("Greedy Ingredients: ",greedySearch(foodEmbedding))

(200,)
Label: ginger_ground
Actual Ingredients:  startseq ginger_was_one_of_the_first_oriental_spices_used_in_the_western_world_when_it_was_brought_from_the_east_by_the_fabled_spice_caravans_for_an_asian_flair mix_ground_ginger_with_soy_sauce toasted_spice_islands_sesame_seeds_and_minced_garlic_drizzle_over_vegetables chicken_or_steak_blend_with_butter_and_brown_sugar_pour_over_cooked_carrots_or_winter_squash_and_broil_use_ground_ginger_to_flavor_marinades curries and_baked_goods endseq
(200,)
(1, 51)


ValueError: Error when checking input: expected input_1 to have shape (200,) but got array with shape (1,)

In [51]:
foodEmbedding.shape

(200,)